In [32]:
# ------------------------- Incidents -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Chargement des 2 dataset
df1 = pd.read_csv(
    "../data/LFB Incident data from 2009 - 2017.csv", sep=",", low_memory=False
)
df2 = pd.read_csv(
    "../data/LFB Incident data from 2018 onwards.csv", sep=",", low_memory=False
)

In [33]:
# concaténation
df = pd.concat([df1, df2], axis=0)
# reset index
df = df.reset_index(drop=True)

# suppression des ".00" sur certaines lignes de "IncidentNumber"
df["IncidentNumber"] = df["IncidentNumber"].apply(
    lambda x: str(x)[0:-3] if (".00" in x) & (str(x)[-3:] == ".00") else x
)

# formatage des dates
df.DateOfCall = pd.to_datetime(df.DateOfCall, format="%d-%b-%y")
df.TimeOfCall = pd.to_datetime(df.TimeOfCall, format="%H:%M:%S")

# création de champs Mois et DayOfWeek
df["Month"] = df["DateOfCall"].dt.month
df["DayOfWeek"] = df["DateOfCall"].dt.dayofweek + 1

display(df.head())
display(df.info())

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Notional Cost (£),NumCalls,Month,DayOfWeek
0,235138081,2009-01-01,2009,1900-01-01 00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,In street close to gazetteer location,SW11 4LB,SW11,NaN,NaN,E09000032,WANDSWORTH,Wandsworth,E05014010,Battersea Park,Battersea Park,528652.0,176830.0,528650,176850,51.475812,-0.148894,London,Battersea,319.0,Battersea,342.0,Clapham,2.0,2.0,2,60,255,1.0,1,4
1,1091,2009-01-01,2009,1900-01-01 00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,Open land/water - nearest gazetteer location,SE1 7SG,SE1,NaN,NaN,E09000022,LAMBETH,Lambeth,E05014118,Waterloo & South Bank,Waterloo & South Bank,530485.0,179007.0,530450,179050,51.494957,-0.121712,London,Lambeth,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0,1,4
2,2091,2009-01-01,2009,1900-01-01 00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,In street outside gazetteer location,N9 9EL,N9,NaN,NaN,E09000010,ENFIELD,Enfield,E05013682,Haselbury,Haselbury,533773.0,194492.0,533750,194450,51.633342,-0.068488,London,Edmonton,308.0,Edmonton,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4
3,3091,2009-01-01,2009,1900-01-01 00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),On land associated with building,UB10 0DG,UB10,1.000215e+11,21401491.0,E09000017,HILLINGDON,Hillingdon,E05013571,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,51.533882,-0.448089,London,Hillingdon,210.0,Hillingdon,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4
4,5091,2009-01-01,2009,1900-01-01 00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,In street outside gazetteer location,N7 8HG,N7,NaN,NaN,E09000019,ISLINGTON,Islington,E05013708,Laycock,Laycock,531058.0,185307.0,531050,185350,51.551441,-0.111120,London,Holloway,233.0,Holloway,250.0,Holloway,1.0,2.0,2,60,255,1.0,1,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 41 columns):
 #   Column                                  Dtype         
---  ------                                  -----         
 0   IncidentNumber                          object        
 1   DateOfCall                              datetime64[ns]
 2   CalYear                                 int64         
 3   TimeOfCall                              datetime64[ns]
 4   HourOfCall                              int64         
 5   IncidentGroup                           object        
 6   StopCodeDescription                     object        
 7   SpecialServiceType                      object        
 8   PropertyCategory                        object        
 9   PropertyType                            object        
 10  AddressQualifier                        object        
 11  Postcode_full                           object        
 12  Postcode_district                       ob

None

In [34]:
def TestDoublonSousCatégorie(categorie, ssCategorie):
    # Vérifie si les StopCodeDescription existe en double dans les IncidentGroup
    result = df.groupby(ssCategorie)[categorie].nunique()
    print(result[result.values > 1])
    print("Na pour", categorie, df[categorie].isna().sum())
    print("Na pour", ssCategorie, df[ssCategorie].isna().sum())

TestDoublonSousCatégorie("IncidentGroup", "StopCodeDescription")
print("\n", "6 na sur IncidentGroup et pas de doublon, peut supprimer la colonne IncidentGroup")

print()
TestDoublonSousCatégorie("PropertyCategory", "PropertyType")
print("\n", "6 na sur PropertyCategory et 6 pour PropertyType, peut supprimer la colonne PropertyCategory")

df.StopCodeDescription = df.StopCodeDescription.str.upper()
df.SpecialServiceType = df.SpecialServiceType.str.upper()
df.PropertyType = df.PropertyType.str.upper()
df.AddressQualifier = df.AddressQualifier.str.upper()
df.Postcode_district = df.Postcode_district.str.upper()
df.IncGeo_BoroughCode = df.IncGeo_BoroughCode.str.upper()
df.IncGeo_WardCode = df.IncGeo_WardCode.str.upper()
df.FirstPumpArriving_DeployedFromStation = df.FirstPumpArriving_DeployedFromStation.str.upper()

Series([], Name: IncidentGroup, dtype: int64)
Na pour IncidentGroup 6
Na pour StopCodeDescription 0

 6 na sur IncidentGroup et pas de doublon, peut supprimer la colonne IncidentGroup

Series([], Name: PropertyCategory, dtype: int64)
Na pour PropertyCategory 6
Na pour PropertyType 6

 6 na sur PropertyCategory et 6 pour PropertyType, peut supprimer la colonne PropertyCategory


In [35]:
# Merge de SpecialService dans Stopcode
s = "Special Service".upper()
df["StopCode"] = df.apply(lambda row: "SST-" + row["SpecialServiceType"] if row["StopCodeDescription"] == s else row["StopCodeDescription"], axis=1)
display(df.head())
# df.StopCodeDescription
# SpecialServiceType

df = df.drop(["StopCodeDescription", "SpecialServiceType"] , axis=1)

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,Latitude,Longitude,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Notional Cost (£),NumCalls,Month,DayOfWeek,StopCode
0,235138081,2009-01-01,2009,1900-01-01 00:00:37,0,Special Service,SPECIAL SERVICE,RTC,Road Vehicle,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11 4LB,SW11,NaN,NaN,E09000032,WANDSWORTH,Wandsworth,E05014010,Battersea Park,Battersea Park,528652.0,176830.0,528650,176850,51.475812,-0.148894,London,Battersea,319.0,BATTERSEA,342.0,Clapham,2.0,2.0,2,60,255,1.0,1,4,SST-RTC
1,1091,2009-01-01,2009,1900-01-01 00:00:46,0,Special Service,SPECIAL SERVICE,ASSIST OTHER AGENCIES,Outdoor,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1 7SG,SE1,NaN,NaN,E09000022,LAMBETH,Lambeth,E05014118,Waterloo & South Bank,Waterloo & South Bank,530485.0,179007.0,530450,179050,51.494957,-0.121712,London,Lambeth,NaN,NaN,NaN,NaN,NaN,NaN,1,60,255,1.0,1,4,SST-ASSIST OTHER AGENCIES
2,2091,2009-01-01,2009,1900-01-01 00:03:00,0,Fire,SECONDARY FIRE,NaN,Outdoor,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9 9EL,N9,NaN,NaN,E09000010,ENFIELD,Enfield,E05013682,Haselbury,Haselbury,533773.0,194492.0,533750,194450,51.633342,-0.068488,London,Edmonton,308.0,EDMONTON,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4,SECONDARY FIRE
3,3091,2009-01-01,2009,1900-01-01 00:04:27,0,Fire,SECONDARY FIRE,NaN,Outdoor,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10 0DG,UB10,1.000215e+11,21401491.0,E09000017,HILLINGDON,Hillingdon,E05013571,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,51.533882,-0.448089,London,Hillingdon,210.0,HILLINGDON,NaN,NaN,1.0,1.0,1,60,255,2.0,1,4,SECONDARY FIRE
4,5091,2009-01-01,2009,1900-01-01 00:05:39,0,Fire,SECONDARY FIRE,NaN,Outdoor,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7 8HG,N7,NaN,NaN,E09000019,ISLINGTON,Islington,E05013708,Laycock,Laycock,531058.0,185307.0,531050,185350,51.551441,-0.111120,London,Holloway,233.0,HOLLOWAY,250.0,Holloway,1.0,2.0,2,60,255,1.0,1,4,SECONDARY FIRE


In [36]:
# suppression des données inutiles, en doublon métier, ou non présente au moment de la prédiction
df = df.drop(
    [
        "DateOfCall",
        "TimeOfCall",
        "IncidentGroup",
        "PropertyCategory",
        #Geo
        "Postcode_full",
        "UPRN",
        "IncGeo_BoroughName",
        "ProperCase",
        "IncGeo_WardName",
        "IncGeo_WardNameNew",
        "Easting_m",
        "Northing_m",
        "Easting_rounded",
        "Northing_rounded",
        "Latitude",
        "Longitude",
        #
        "FRS",
        "IncidentStationGround",
        "FirstPumpArriving_AttendanceTime",
        "SecondPumpArriving_AttendanceTime",
        "SecondPumpArriving_DeployedFromStation",
        "NumStationsWithPumpsAttending",
        "PumpCount",
        "PumpMinutesRounded",
        "Notional Cost (£)",
        "NumCalls",
    ],
    axis=1,
)

In [37]:
# sauvegarde d'un fichier temporaire
df.to_csv("../data/PreIncidents.csv", sep=";", index=False) 

In [38]:
# ------------------------- Mobilisations -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Chargement des 3 dataset
df1 = pd.read_csv(
    "../data/LFB Mobilisation data from January 2009 - 2014.csv",
    sep=";",
    low_memory=False,
)
df2 = pd.read_csv(
    "../data/LFB Mobilisation data from 2015 - 2020.csv", sep=";", low_memory=False
)
df3 = pd.read_csv(
    "../data/LFB Mobilisation data from 2021 - 2024.csv",
    sep=",",
    low_memory=False,
    usecols=lambda column: column not in ["BoroughName", "WardName"],
)

In [39]:
# concaténation
df = pd.concat([df1, df2, df3], axis=0)
# reset index
df = df.reset_index(drop=True)

# formatage des dates
df.DateAndTimeMobilised = pd.to_datetime(df.DateAndTimeMobilised, format='%d/%m/%Y %H:%M')
df.DateAndTimeMobile = pd.to_datetime(df.DateAndTimeMobile, format="%d/%m/%Y %H:%M")
df.DateAndTimeArrived = pd.to_datetime(df.DateAndTimeArrived, format="%d/%m/%Y %H:%M")
df.DateAndTimeLeft = pd.to_datetime(df.DateAndTimeLeft, format="%d/%m/%Y %H:%M")
df.DateAndTimeReturned = pd.to_datetime(df.DateAndTimeReturned, format="%d/%m/%Y %H:%M")

display(df.head())
display(df.info())

,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,DateAndTimeArrived,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description
0,235138081,2009,0,38426,H271,1,2009-01-01 00:02:00,NaT,2009-01-01 00:07:00,NaN,NaN,319,2009-01-01 00:16:00,2009-01-01 00:23:00,H27,Battersea,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
1,235138081,2009,0,38427,H212,2,2009-01-01 00:02:00,2009-01-01 00:06:00,2009-01-01 00:08:00,253.0,89.0,342,2009-01-01 00:12:00,2009-01-01 00:12:00,H21,Clapham,Home Station,2,Initial,Initial Mobilisation,NaN,NaN
2,2091,2009,0,38429,A341,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:09:00,151.0,157.0,308,2009-01-01 00:16:00,2009-01-01 00:17:00,A34,Edmonton,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
3,3091,2009,0,38430,G232,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:08:00,108.0,102.0,210,2009-01-01 00:20:00,2009-01-01 00:20:00,G23,Hillingdon,Home Station,1,Initial,Initial Mobilisation,NaN,NaN
4,5091,2009,0,38432,A311,1,2009-01-01 00:06:00,2009-01-01 00:07:00,2009-01-01 00:09:00,114.0,119.0,233,2009-01-01 00:11:00,2009-01-01 00:22:00,A31,Holloway,Home Station,1,Initial,Initial Mobilisation,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480004 entries, 0 to 2480003
Data columns (total 22 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   IncidentNumber            object        
 1   CalYear                   int64         
 2   HourOfCall                int64         
 3   ResourceMobilisationId    int64         
 4   Resource_Code             object        
 5   PerformanceReporting      object        
 6   DateAndTimeMobilised      datetime64[ns]
 7   DateAndTimeMobile         datetime64[ns]
 8   DateAndTimeArrived        datetime64[ns]
 9   TurnoutTimeSeconds        float64       
 10  TravelTimeSeconds         float64       
 11  AttendanceTimeSeconds     int64         
 12  DateAndTimeLeft           datetime64[ns]
 13  DateAndTimeReturned       datetime64[ns]
 14  DeployedFromStation_Code  object        
 15  DeployedFromStation_Name  object        
 16  DeployedFromLocation      object        
 17  PumpOrde

None

In [40]:
# création de la colonne Temps sur site en seconde de la pompe par delta de temps arrivée et départ
df["PumpOnSiteSeconds"] = (df.DateAndTimeLeft - df.DateAndTimeArrived).dt.seconds
display(df.head(3))

# Regroupe les lignes par incidents, car il y a une ligne par incidents / camion avec un n° d'ordre
# Comme on s'intéresse à prédire des temps, on calcule les temps par incidents, min, max et moyen
# et on cherchera à prédire des tgemps
# les infos de 1ere station sur place et nb de pompe seront prises à partir de incidents
aggregated = (
    df.groupby("IncidentNumber")
    .agg(
        PumpSecondsOnSite_min=("PumpOnSiteSeconds", "min"),
        PumpSecondsOnSite_mean=("PumpOnSiteSeconds", "mean"),
        PumpSecondsOnSite_max=("PumpOnSiteSeconds", "max"),
        TurnoutTimeSeconds_min=("TurnoutTimeSeconds", "min"),
        TurnoutTimeSeconds_mean=("TurnoutTimeSeconds", "mean"),
        TurnoutTimeSeconds_max=("TurnoutTimeSeconds", "max"),
        TravelTimeSeconds_min=("TravelTimeSeconds", "min"),
        TravelTimeSeconds_mean=("TravelTimeSeconds", "mean"),
        TravelTimeSeconds_max=("TravelTimeSeconds", "max"),
    )    
    .reset_index()
)
display(aggregated.head())

# jointure de l'agrégat sur le dataframe pour ajouter les colonnes de temps
merge = pd.merge(
    df, aggregated, left_on="IncidentNumber", right_on="IncidentNumber", how="left"
).sort_values(by=["IncidentNumber", "PumpOrder"])

display(merge.head(10))
print(merge.shape)

,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,DateAndTimeArrived,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description,PumpOnSiteSeconds
0,235138081,2009,0,38426,H271,1,2009-01-01 00:02:00,NaT,2009-01-01 00:07:00,NaN,NaN,319,2009-01-01 00:16:00,2009-01-01 00:23:00,H27,Battersea,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,540.0
1,235138081,2009,0,38427,H212,2,2009-01-01 00:02:00,2009-01-01 00:06:00,2009-01-01 00:08:00,253.0,89.0,342,2009-01-01 00:12:00,2009-01-01 00:12:00,H21,Clapham,Home Station,2,Initial,Initial Mobilisation,NaN,NaN,240.0
2,2091,2009,0,38429,A341,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:09:00,151.0,157.0,308,2009-01-01 00:16:00,2009-01-01 00:17:00,A34,Edmonton,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,420.0


,IncidentNumber,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1092,NaN,NaN,NaN,128.0,128.0,128.0,390.0,390.0,390.0
1,1111,120.0,120.0,120.0,71.0,71.0,71.0,480.0,480.0,480.0
2,1112,480.0,480.0,480.0,119.0,119.0,119.0,225.0,225.0,225.0
3,1121,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
4,1131,360.0,360.0,360.0,89.0,89.0,89.0,78.0,78.0,78.0


,IncidentNumber,CalYear,HourOfCall,ResourceMobilisationId,Resource_Code,PerformanceReporting,DateAndTimeMobilised,DateAndTimeMobile,DateAndTimeArrived,TurnoutTimeSeconds,TravelTimeSeconds,AttendanceTimeSeconds,DateAndTimeLeft,DateAndTimeReturned,DeployedFromStation_Code,DeployedFromStation_Name,DeployedFromLocation,PumpOrder,PlusCode_Code,PlusCode_Description,DelayCodeId,DelayCode_Description,PumpOnSiteSeconds,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
25229,1092,2009,10,72977,G221,1,2009-02-23 10:02:00,2009-02-23 10:04:00,2009-02-23 10:11:00,128.0,390.0,518,NaT,2009-02-23 10:22:00,G22,Stanmore,Home Station,1,Initial,Initial Mobilisation,12.0,Not held up,NaN,NaN,NaN,NaN,128.0,128.0,128.0,390.0,390.0,390.0
338999,1111,2011,0,3610263,A411,1,2011-01-01 00:03:00,2011-01-01 00:04:00,2011-01-01 00:12:00,71.0,480.0,551,2011-01-01 00:14:00,2011-01-01 00:20:00,A41,West Hampstead,Home Station,1,Initial,Initial Mobilisation,12.0,Not held up,120.0,120.0,120.0,120.0,71.0,71.0,71.0,480.0,480.0,480.0
346998,1112,2011,1,3621855,E331,1,2011-01-22 01:32:00,2011-01-22 01:34:00,2011-01-22 01:38:00,119.0,225.0,344,2011-01-22 01:46:00,2011-01-22 01:47:00,E33,Southwark,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,480.0,480.0,480.0,480.0,119.0,119.0,119.0,225.0,225.0,225.0
492154,1121,2012,0,3831037,G401,1,2012-01-01 00:01:00,2012-01-01 00:02:00,2012-01-01 00:05:00,65.0,192.0,257,2012-01-01 00:11:00,2012-01-01 00:18:00,G40,Hayes,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,360.0,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
492155,1121,2012,0,3831038,G402,2,2012-01-01 00:01:00,2012-01-01 00:02:00,2012-01-01 00:06:00,67.0,235.0,302,2012-01-01 00:10:00,2012-01-01 00:17:00,G40,Hayes,Home Station,2,Initial,Initial Mobilisation,NaN,NaN,240.0,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
637588,1131,2013,0,4036113,A412,1,2013-01-01 00:02:00,2013-01-01 00:03:00,2013-01-01 00:05:00,89.0,78.0,167,2013-01-01 00:11:00,2013-01-01 00:12:00,A41,West Hampstead,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,360.0,360.0,360.0,360.0,89.0,89.0,89.0,78.0,78.0,78.0
733568,1133,2013,12,4179294,G281,1,2013-09-10 11:06:00,2013-09-10 11:06:00,2013-09-10 11:11:00,1.0,267.0,268,2013-09-10 11:14:00,2013-09-10 11:14:00,G28,Willesden,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,180.0,180.0,180.0,180.0,1.0,1.0,1.0,267.0,267.0,267.0
880681,1143,2014,11,4407663,H431,1,2014-10-28 11:57:00,2014-10-28 11:58:00,2014-10-28 12:04:00,42.0,377.0,419,2014-10-28 12:08:00,NaT,H43,Twickenham,Home Station,1,Initial,Initial Mobilisation,9.0,"Traffic, roadworks, etc",240.0,240.0,240.0,240.0,42.0,42.0,42.0,377.0,377.0,377.0
2,2091,2009,0,38429,A341,1,2009-01-01 00:04:00,2009-01-01 00:06:00,2009-01-01 00:09:00,151.0,157.0,308,2009-01-01 00:16:00,2009-01-01 00:17:00,A34,Edmonton,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,420.0,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
25230,2092,2009,10,72978,E241,1,2009-02-23 10:03:00,2009-02-23 10:04:00,2009-02-23 10:06:00,85.0,85.0,170,NaT,2009-02-23 10:17:00,E24,Woolwich,Home Station,1,Initial,Initial Mobilisation,NaN,NaN,NaN,300.0,300.0,300.0,78.0,81.5,85.0,85.0,208.0,331.0


(2480004, 32)


In [41]:
# Pas de traitement particulier sur PlusCode_Code, car les lignes Add et RCA ne sont pas des lignes en doublons d'un Initial, et peuvent donc être traitées comme initial

# suppression des duplicatas pour ne garder qu'une ligne par incidents
merge = merge.drop_duplicates(subset=["IncidentNumber"], keep="first")
merge.IncidentNumber.duplicated().sum()

np.int64(0)

In [42]:
# suppression des données inutiles, en doublon métier, ou non présente au moment de la prédiction
merge = merge.drop(
    [
        "CalYear",
        "HourOfCall",
        "ResourceMobilisationId",
        "Resource_Code",
        "PerformanceReporting",
        "DateAndTimeMobilised",
        "DateAndTimeMobile",
        "DateAndTimeArrived",
        "TurnoutTimeSeconds",
        "TravelTimeSeconds",
        "PumpOnSiteSeconds",
        "AttendanceTimeSeconds",
        "DateAndTimeLeft",
        "DateAndTimeReturned",
        "DeployedFromStation_Code",
        "DeployedFromStation_Name",
        "DeployedFromLocation",
        "PumpOrder",
        "PlusCode_Code",
        "PlusCode_Description",
        "DelayCodeId",
        "DelayCode_Description",
    ],
    axis=1,
)

In [43]:
# sauvegarde d'un fichier temporaire
merge.to_csv("../data/PreMobilisations.csv", sep=";", index=False)

In [44]:
# ------------------------- Merge Incidents et Mobilisations -----------------------------
import pandas as pd

# Ajuster les paramètres pour afficher toutes les lignes et colonnes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Charge les dataset temporaire
df_incidents = pd.read_csv(
    "../data/PreIncidents.csv", sep=";", low_memory=False
)
df_mobilisations = pd.read_csv(
    "../data/PreMobilisations.csv", sep=";", low_memory=False
)

display(df_incidents.head())
display(df_incidents.info())

display(df_mobilisations.head())
display(df_mobilisations.info())

,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode
0,235138081,2009,0,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11,NaN,E09000032,E05014010,BATTERSEA,2.0,1,4,SST-RTC
1,1091,2009,0,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1,NaN,E09000022,E05014118,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES
2,2091,2009,0,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9,NaN,E09000010,E05013682,EDMONTON,1.0,1,4,SECONDARY FIRE
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10,21401491.0,E09000017,E05013571,HILLINGDON,1.0,1,4,SECONDARY FIRE
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7,NaN,E09000019,E05013708,HOLLOWAY,2.0,1,4,SECONDARY FIRE


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 14 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   IncidentNumber                         object 
 1   CalYear                                int64  
 2   HourOfCall                             int64  
 3   PropertyType                           object 
 4   AddressQualifier                       object 
 5   Postcode_district                      object 
 6   USRN                                   float64
 7   IncGeo_BoroughCode                     object 
 8   IncGeo_WardCode                        object 
 9   FirstPumpArriving_DeployedFromStation  object 
 10  NumPumpsAttending                      float64
 11  Month                                  int64  
 12  DayOfWeek                              int64  
 13  StopCode                               object 
dtypes: float64(2), int64(4), object(8)
memory usage: 1

None

,IncidentNumber,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,1092,NaN,NaN,NaN,128.0,128.0,128.0,390.0,390.0,390.0
1,1111,120.0,120.0,120.0,71.0,71.0,71.0,480.0,480.0,480.0
2,1112,480.0,480.0,480.0,119.0,119.0,119.0,225.0,225.0,225.0
3,1121,240.0,300.0,360.0,65.0,66.0,67.0,192.0,213.5,235.0
4,1131,360.0,360.0,360.0,89.0,89.0,89.0,78.0,78.0,78.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642042 entries, 0 to 1642041
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   IncidentNumber           1642042 non-null  object 
 1   PumpSecondsOnSite_min    1612710 non-null  float64
 2   PumpSecondsOnSite_mean   1612710 non-null  float64
 3   PumpSecondsOnSite_max    1612710 non-null  float64
 4   TurnoutTimeSeconds_min   1624689 non-null  float64
 5   TurnoutTimeSeconds_mean  1624689 non-null  float64
 6   TurnoutTimeSeconds_max   1624689 non-null  float64
 7   TravelTimeSeconds_min    1624692 non-null  float64
 8   TravelTimeSeconds_mean   1624692 non-null  float64
 9   TravelTimeSeconds_max    1624692 non-null  float64
dtypes: float64(9), object(1)
memory usage: 125.3+ MB


None

In [45]:
# Merge
df = pd.merge(
    df_incidents,
    df_mobilisations,
    left_on="IncidentNumber",
    right_on="IncidentNumber",
    how="left",
)
df = df.reset_index(drop=True)

# on vérifie qu'il n'y a pas de doublon sur IncidentNumber avant de la supprimer
print("IncidentNumber na :", df.IncidentNumber.isna().sum())

display(df.head())
display(df.info())

IncidentNumber na : 0


,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,235138081,2009,0,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11,NaN,E09000032,E05014010,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1091,2009,0,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1,NaN,E09000022,E05014118,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009,0,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9,NaN,E09000010,E05013682,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10,21401491.0,E09000017,E05013571,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7,NaN,E09000019,E05013708,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 23 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   IncidentNumber                         object 
 1   CalYear                                int64  
 2   HourOfCall                             int64  
 3   PropertyType                           object 
 4   AddressQualifier                       object 
 5   Postcode_district                      object 
 6   USRN                                   float64
 7   IncGeo_BoroughCode                     object 
 8   IncGeo_WardCode                        object 
 9   FirstPumpArriving_DeployedFromStation  object 
 10  NumPumpsAttending                      float64
 11  Month                                  int64  
 12  DayOfWeek                              int64  
 13  StopCode                               object 
 14  PumpSecondsOnSite_min                  float64
 15

None

In [46]:
# sauvegarde d'un fichier temporaire
df.to_csv("../data/PreProcessTemp.csv", sep=";", index=False)

In [55]:
# Nettoyage du fichier PreProcessTemp. Pas de doublon sur IncidentNumber, vérifié juste avant
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/PreProcessTemp.csv", sep=";", low_memory=False)

display(df.head())
display(df.info())


,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,235138081,2009,0,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11,NaN,E09000032,E05014010,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1091,2009,0,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1,NaN,E09000022,E05014118,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009,0,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9,NaN,E09000010,E05013682,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10,21401491.0,E09000017,E05013571,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7,NaN,E09000019,E05013708,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1771413 entries, 0 to 1771412
Data columns (total 23 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   IncidentNumber                         object 
 1   CalYear                                int64  
 2   HourOfCall                             int64  
 3   PropertyType                           object 
 4   AddressQualifier                       object 
 5   Postcode_district                      object 
 6   USRN                                   float64
 7   IncGeo_BoroughCode                     object 
 8   IncGeo_WardCode                        object 
 9   FirstPumpArriving_DeployedFromStation  object 
 10  NumPumpsAttending                      float64
 11  Month                                  int64  
 12  DayOfWeek                              int64  
 13  StopCode                               object 
 14  PumpSecondsOnSite_min                  float64
 15

None

In [56]:
# Gestion des NA

def ShowNa():
    # Affichage des NA
    na_counts = df.isna().sum()
    print(na_counts[na_counts > 0],"\n")

    print("\ntotal ligne na", df.isna().any(axis=1).sum())
    print("Total de lignes", len(df))
    print("Poucentage", df.isna().any(axis=1).sum() / len(df) * 100, "%")

    print("\nligne par année")
    # display(df.CalYear.value_counts())

    # print("ligne na par année")
    # display(df_temp.loc[df_temp.isna().any(axis=1), "CalYear"].value_counts())
    # print("Les lignes avec des na sont principalments des lignes d'avant 2014")

    # years = []
    # for year in range(df_temp.CalYear.min(), df_temp.CalYear.max() + 1):
    #     total = len(df_temp.loc[df_temp.CalYear == year])
    #     na = df_temp[df_temp.CalYear == year].isna().any(axis=1).sum()
    #     years.append([year, total, na, na / total * 100])

    years = []
    for year in range(df.CalYear.min(), df.CalYear.max() + 1):
        for col in df.columns:
            total = len(df.loc[df.CalYear == year])
            na = df.loc[df.CalYear == year, col].isna().sum()
            if na > 0:
                years.append([year, col, total, na, na / total * 100])

    df_years = pd.DataFrame(years, columns=["Année", "Colonne", "Nb lignes", "Na", "%Na"])
    # display(df_years)
    display(df_years.groupby(["Année", "Colonne"]).min())
    display(df.loc[df.USRN.isna()].head(5))


ShowNa()

# supprime les lignes avec très peu de NA
# les lignes avec des USRS vides datent d'avant 2015, on les supprime
df = df.dropna(axis = 0, how = 'any', subset =["PropertyType", "AddressQualifier", "IncGeo_WardCode", "NumPumpsAttending", "USRN"])
ShowNa()

# Il n'existe pas toujours de ligne d'intervention pour les lignes d'Incidents, on supprime les lignes sans informations de mobilisation, et parfois l'information est partiellement saisie
df = df.dropna(axis=0, how="any", subset=["PumpSecondsOnSite_min", "TurnoutTimeSeconds_min", "TravelTimeSeconds_min", "FirstPumpArriving_DeployedFromStation"])
ShowNa()


PropertyType                                  6
AddressQualifier                              1
USRN                                     162871
IncGeo_WardCode                             515
FirstPumpArriving_DeployedFromStation    135700
NumPumpsAttending                         14103
PumpSecondsOnSite_min                    165344
PumpSecondsOnSite_mean                   165344
PumpSecondsOnSite_max                    165344
TurnoutTimeSeconds_min                   153350
TurnoutTimeSeconds_mean                  153350
TurnoutTimeSeconds_max                   153350
TravelTimeSeconds_min                    153347
TravelTimeSeconds_mean                   153347
TravelTimeSeconds_max                    153347
dtype: int64 


total ligne na 316080
Total de lignes 1771413
Poucentage 17.843382655541085 %

ligne par année


Nb lignes     Na        %Na
Année Colonne                                                           
2009  FirstPumpArriving_DeployedFromStation     134379  13173   9.802871
      NumPumpsAttending                         134379   1341   0.997924
      PumpSecondsOnSite_max                     134379  33117  24.644476
      PumpSecondsOnSite_mean                    134379  33117  24.644476
      PumpSecondsOnSite_min                     134379  33117  24.644476
      TravelTimeSeconds_max                     134379  17350  12.911244
      TravelTimeSeconds_mean                    134379  17350  12.911244
      TravelTimeSeconds_min                     134379  17350  12.911244
      TurnoutTimeSeconds_max                    134379  17349  12.910499
      TurnoutTimeSeconds_mean                   134379  17349  12.910499
      TurnoutTimeSeconds_min                    134379  17349  12.910499
      USRN                                      134379  29464  21.926045
2010  FirstPumpArriving_DeployedFromStation     125060  14704  11.757556
      NumPumpsAttending                         125060    672   0.537342
      PumpSecondsOnSite_max                     125060  18337  14.662562
      PumpSecondsOnSite_mean                    125060  18337  14.662562
      PumpSecondsOnSite_min                     125060  18337  14.662562
      TravelTimeSeconds_max                     125060  16541  13.226451
      TravelTimeSeconds_mean                    125060  16541  13.226451
      TravelTimeSeconds_min                     125060  16541  13.226451
      TurnoutTimeSeconds_max                    125060  16556  13.238446
      TurnoutTimeSeconds_mean                   125060  16556  13.238446
      TurnoutTimeSeconds_min                    125060  16556  13.238446
      USRN                                      125060  26515  21.201823
2011  FirstPumpArriving_DeployedFromStation     115132  12448  10.811938
      NumPumpsAttending                         115132    447   0.388250
      PumpSecondsOnSite_max                     115132  15613  13.560956
      PumpSecondsOnSite_mean                    115132  15613  13.560956
      PumpSecondsOnSite_min                     115132  15613  13.560956
      TravelTimeSeconds_max                     115132  14014  12.172115
      TravelTimeSeconds_mean                    115132  14014  12.172115
      TravelTimeSeconds_min                     115132  14014  12.172115
      TurnoutTimeSeconds_max                    115132  14012  12.170378
      TurnoutTimeSeconds_mean                   115132  14012  12.170378
      TurnoutTimeSeconds_min                    115132  14012  12.170378
      USRN                                      115132  24788  21.530070
2012  FirstPumpArriving_DeployedFromStation     108391  11612  10.713067
      NumPumpsAttending                         108391    746   0.688249
      PumpSecondsOnSite_max                     108391  12348  11.392090
      PumpSecondsOnSite_mean                    108391  12348  11.392090
      PumpSecondsOnSite_min                     108391  12348  11.392090
      TravelTimeSeconds_max                     108391  12977  11.972396
      TravelTimeSeconds_mean                    108391  12977  11.972396
      TravelTimeSeconds_min                     108391  12977  11.972396
      TurnoutTimeSeconds_max                    108391  12971  11.966861
      TurnoutTimeSeconds_mean                   108391  12971  11.966861
      TurnoutTimeSeconds_min                    108391  12971  11.966861
      USRN                                      108391  21198  19.556974
2013  FirstPumpArriving_DeployedFromStation     103055   9933   9.638543
      NumPumpsAttending                         103055    381   0.369705
      PumpSecondsOnSite_max                     103055  10538  10.225608
      PumpSecondsOnSite_mean                    103055  10538  10.225608
      PumpSecondsOnSite_min                     103055  10538  10.225608
      TravelTimeSeconds_max   

,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
0,235138081,2009,0,CAR,IN STREET CLOSE TO GAZETTEER LOCATION,SW11,NaN,E09000032,E05014010,BATTERSEA,2.0,1,4,SST-RTC,240.0,390.0,540.0,253.0,253.0,253.0,89.0,89.0,89.0
1,1091,2009,0,LAKE/POND/RESERVOIR,OPEN LAND/WATER - NEAREST GAZETTEER LOCATION,SE1,NaN,E09000022,E05014118,NaN,NaN,1,4,SST-ASSIST OTHER AGENCIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009,0,ROAD SURFACE/PAVEMENT,IN STREET OUTSIDE GAZETTEER LOCATION,N9,NaN,E09000010,E05013682,EDMONTON,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,151.0,151.0,151.0,157.0,157.0,157.0
4,5091,2009,0,CYCLE PATH/PUBLIC FOOTPATH/BRIDLEWAY,IN STREET OUTSIDE GAZETTEER LOCATION,N7,NaN,E09000019,E05013708,HOLLOWAY,2.0,1,4,SECONDARY FIRE,120.0,120.0,120.0,114.0,128.0,142.0,108.0,113.5,119.0
6,8091,2009,0,CAR,ON MOTORWAY / ELEVATED ROAD,SE3,NaN,E09000011,E05014084,EAST GREENWICH,1.0,1,4,SST-RTC,420.0,420.0,420.0,119.0,119.0,119.0,403.0,403.0,403.0


FirstPumpArriving_DeployedFromStation    107814
PumpSecondsOnSite_min                    129492
PumpSecondsOnSite_mean                   129492
PumpSecondsOnSite_max                    129492
TurnoutTimeSeconds_min                   120723
TurnoutTimeSeconds_mean                  120723
TurnoutTimeSeconds_max                   120723
TravelTimeSeconds_min                    120745
TravelTimeSeconds_mean                   120745
TravelTimeSeconds_max                    120745
dtype: int64 


total ligne na 140292
Total de lignes 1595625
Poucentage 8.792291421856639 %

ligne par année


Nb lignes     Na        %Na
Année Colonne                                                           
2009  FirstPumpArriving_DeployedFromStation     103914   8776   8.445445
      PumpSecondsOnSite_max                     103914  23562  22.674519
      PumpSecondsOnSite_mean                    103914  23562  22.674519
      PumpSecondsOnSite_min                     103914  23562  22.674519
      TravelTimeSeconds_max                     103914  11648  11.209269
      TravelTimeSeconds_mean                    103914  11648  11.209269
      TravelTimeSeconds_min                     103914  11648  11.209269
      TurnoutTimeSeconds_max                    103914  11646  11.207345
      TurnoutTimeSeconds_mean                   103914  11646  11.207345
      TurnoutTimeSeconds_min                    103914  11646  11.207345
2010  FirstPumpArriving_DeployedFromStation      98097  11545  11.768963
      PumpSecondsOnSite_max                      98097  14048  14.320519
      PumpSecondsOnSite_mean                     98097  14048  14.320519
      PumpSecondsOnSite_min                      98097  14048  14.320519
      TravelTimeSeconds_max                      98097  12621  12.865837
      TravelTimeSeconds_mean                     98097  12621  12.865837
      TravelTimeSeconds_min                      98097  12621  12.865837
      TurnoutTimeSeconds_max                     98097  12630  12.875011
      TurnoutTimeSeconds_mean                    98097  12630  12.875011
      TurnoutTimeSeconds_min                     98097  12630  12.875011
2011  FirstPumpArriving_DeployedFromStation      90077   9876  10.963953
      PumpSecondsOnSite_max                      90077  12047  13.374113
      PumpSecondsOnSite_mean                     90077  12047  13.374113
      PumpSecondsOnSite_min                      90077  12047  13.374113
      TravelTimeSeconds_max                      90077  10749  11.933124
      TravelTimeSeconds_mean                     90077  10749  11.933124
      TravelTimeSeconds_min                      90077  10749  11.933124
      TurnoutTimeSeconds_max                     90077  10745  11.928683
      TurnoutTimeSeconds_mean                    90077  10745  11.928683
      TurnoutTimeSeconds_min                     90077  10745  11.928683
2012  FirstPumpArriving_DeployedFromStation      86837   9136  10.520861
      PumpSecondsOnSite_max                      86837   9647  11.109320
      PumpSecondsOnSite_mean                     86837   9647  11.109320
      PumpSecondsOnSite_min                      86837   9647  11.109320
      TravelTimeSeconds_max                      86837   9883  11.381093
      TravelTimeSeconds_mean                     86837   9883  11.381093
      TravelTimeSeconds_min                      86837   9883  11.381093
      TurnoutTimeSeconds_max                     86837   9876  11.373032
      TurnoutTimeSeconds_mean                    86837   9876  11.373032
      TurnoutTimeSeconds_min                     86837   9876  11.373032
2013  FirstPumpArriving_DeployedFromStation      82857   8027   9.687775
      PumpSecondsOnSite_max                      82857   8443  10.189845
      PumpSecondsOnSite_mean                     82857   8443  10.189845
      PumpSecondsOnSite_min                      82857   8443  10.189845
      TravelTimeSeconds_max                      82857   8653  10.443294
      TravelTimeSeconds_mean                     82857   8653  10.443294
      TravelTimeSeconds_min                      82857   8653  10.443294
      TurnoutTimeSeconds_max                     82857   8661  10.452949
      TurnoutTimeSeconds_mean                    82857   8661  10.452949
      TurnoutTimeSeconds_min                     82857   8661  10.452949
2014  FirstPumpArriving_DeployedFromStation      77455   6479   8.364857
      PumpSecondsOnSite_max                      77455   6789   8.765089
      PumpSecondsOnSite_mean                     77455   6789   8.765089
      PumpSecondsOnSite_min   

,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max


Series([], dtype: int64) 


total ligne na 0
Total de lignes 1455333
Poucentage 0.0 %

ligne par année


,,Nb lignes,Na,%Na
Année,Colonne,,,


,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max


In [57]:
df.head()

,IncidentNumber,CalYear,HourOfCall,PropertyType,AddressQualifier,Postcode_district,USRN,IncGeo_BoroughCode,IncGeo_WardCode,FirstPumpArriving_DeployedFromStation,NumPumpsAttending,Month,DayOfWeek,StopCode,PumpSecondsOnSite_min,PumpSecondsOnSite_mean,PumpSecondsOnSite_max,TurnoutTimeSeconds_min,TurnoutTimeSeconds_mean,TurnoutTimeSeconds_max,TravelTimeSeconds_min,TravelTimeSeconds_mean,TravelTimeSeconds_max
3,3091,2009,0,DOMESTIC GARDEN (VEGETATION NOT EQUIPMENT),ON LAND ASSOCIATED WITH BUILDING,UB10,21401491.0,E09000017,E05013571,HILLINGDON,1.0,1,4,SECONDARY FIRE,720.0,720.0,720.0,108.0,108.0,108.0,102.0,102.0,102.0
5,6091,2009,0,PURPOSE BUILT FLATS/MAISONETTES - UP TO 3 STOR...,CORRECT INCIDENT LOCATION,NW5,20499122.0,E09000007,E05013663,KENTISH TOWN,2.0,1,4,AFA,360.0,360.0,360.0,83.0,89.0,95.0,89.0,108.0,127.0
7,9091,2009,0,REFUSE/RUBBISH TIP,ON LAND ASSOCIATED WITH BUILDING,TW3,21500957.0,E09000018,E05013623,HESTON,2.0,1,4,SECONDARY FIRE,1440.0,1440.0,1440.0,178.0,180.5,183.0,164.0,165.0,166.0
8,10091,2009,0,SMALL REFUSE/RUBBISH CONTAINER,ON LAND ASSOCIATED WITH BUILDING,EC1V,21604971.0,E09000019,E05013711,CLERKENWELL,1.0,1,4,SECONDARY FIRE,420.0,420.0,420.0,121.0,121.0,121.0,134.0,134.0,134.0
9,11091,2009,0,STUDENT HALL OF RESIDENCE,CORRECT INCIDENT LOCATION,WC1B,20401090.0,E09000007,E05013653,EUSTON,1.0,1,4,AFA,780.0,780.0,780.0,110.0,110.0,110.0,187.0,187.0,187.0


In [ ]:
# Traitmeent des catégories. Uniquement des catégories nominales
# possibilité Test Anova,


def ShowValues(col):
    print(col, len(df[col].unique()))
    # print(df[col].value_counts())


cols_categ = [
    "StopCode",
    "PropertyType",
    "AddressQualifier",
    "Postcode_district",
    "USRN",
    "IncGeo_BoroughCode",
    "IncGeo_WardCode",
    "FirstPumpArriving_DeployedFromStation",
]

cols_cible = [
    "PumpSecondsOnSite_min",
    "PumpSecondsOnSite_mean",
    "PumpSecondsOnSite_max",
    "TurnoutTimeSeconds_min",
    "TurnoutTimeSeconds_mean",
    "TurnoutTimeSeconds_max",
    "TravelTimeSeconds_min",
    "TravelTimeSeconds_mean",
    "TravelTimeSeconds_max",
]

cols_cible_cat = ["FirstPumpArriving_DeployedFromStation", "NumPumpsAttending"]

# prépare les X et y de tests
# X = df.drop(cols_cible, axis=1)
# list_y = []
# for col in cols_cible:
#     list_y.append([col, df[col]])

# nb de valeurs par catégories
for col in cols_categ:
    ShowValues(col)
print()

df_temp = df[df.CalYear > 2023]

# Sélection de caractéristiques récursive
import statsmodels.api
from statsmodels.api import stats
import statsmodels.formula.api as smf

# Anova sur chaque colonne cible avec toutes les colonnes catégorielles
all_result = []
cols_to_check = "+".join(cols_categ)
for col in cols_cible:
    print("Test de", col)
    results = smf.ols(f"{col} ~ {cols_to_check}", data=df_temp).fit()
    # table = stats.anova_lm(result)
    # display(table)
    results_df = pd.DataFrame(
        {f"{col} coefficients": round(results.params, 3), f"{col} pvalues": round(results.pvalues, 3)}
    )
    # display(results_df.head(30))
    # print("Test de", col, "ok")
    all_result.append(results_df)

# merge les result dans un dataFrame
compiled_results = all_result[0]
i = 0
for result in all_result:
    if i == 0:
        print("continue")
        i += 1
        continue
    print("merge")
    compiled_results = pd.merge(
        compiled_results, result, left_index=True, right_index=True, how="outer"
    )

display(compiled_results.head(3))

# regarde si la pvalue est la même pour toutes les caétgories : ajout de la colonne Discord : True si il y a des delta
# filtre les lignes avec que des valeurs > 0.05 ou que que des valeurs < 0.05
# pour voir si certaines interviennent que dans certains cas
def TreatRow(row):
    keeps = []
    nb = 0
    for i in range(1, len(compiled_results.columns), 2):
        # print(compiled_results.columns[i])
        keeps.append(row[compiled_results.columns[i]] <= 0.05)
        nb += 1
    val = not (sum(keeps) == 0) | (sum(keeps) == nb)  # sum(keeps)
    # print(val)
    return val

compiled_results["Discord"] = compiled_results.apply(lambda row: TreatRow(row), axis=1)
display(compiled_results.head())

print(f"{compiled_results["Discord"].sum()} discords sur {len(compiled_results)}")
#957 discords sur 1440


# pour les temps de mobilisation on considère
# model = smf.ols(
#     formula="TravelTimeSeconds_min ~ StopCode + PropertyType + USRN + HourOfCall",
#     data=df_temp,
# )
# results = model.fit()
# # Résumé des résultats
# display(results.summary())

# model = smf.ols(
#     formula="PumpSecondsOnSite_min ~ StopCode + PropertyType + USRN",
#     data=df_temp,
# )
# results = model.fit()
# results_df = pd.DataFrame({"coefficients": results.params, "pvalues": results.pvalues})
# # Filtrer pour ne garder que les p-values <= 0.05
# significant_results = results_df[results_df["pvalues"] <= 0.05]
# # Afficher les résultats filtrés
# display(significant_results)


StopCode 28
PropertyType 291
AddressQualifier 11
Postcode_district 322
USRN 53339
IncGeo_BoroughCode 33
IncGeo_WardCode 843
FirstPumpArriving_DeployedFromStation 117

continue
merge
merge
merge
merge
merge
merge
merge
merge


,PumpSecondsOnSite_min coefficients,PumpSecondsOnSite_min pvalues,PumpSecondsOnSite_mean coefficients,PumpSecondsOnSite_mean pvalues,PumpSecondsOnSite_max coefficients,PumpSecondsOnSite_max pvalues,TurnoutTimeSeconds_min coefficients,TurnoutTimeSeconds_min pvalues,TurnoutTimeSeconds_mean coefficients,TurnoutTimeSeconds_mean pvalues,TurnoutTimeSeconds_max coefficients,TurnoutTimeSeconds_max pvalues,TravelTimeSeconds_min coefficients,TravelTimeSeconds_min pvalues,TravelTimeSeconds_mean coefficients,TravelTimeSeconds_mean pvalues,TravelTimeSeconds_max coefficients,TravelTimeSeconds_max pvalues
AddressQualifier[T.IN STREET CLOSE TO GAZETTEER LOCATION],-71.314981,0.094,-111.342187,0.011,-156.308071,0.001,-1.701157,0.188,-2.016420,0.120,-2.592226,0.093,-11.171112,0.006,-11.915869,0.005,-13.563712,0.006
AddressQualifier[T.IN STREET OUTSIDE GAZETTEER LOCATION],-32.952347,0.389,-76.945215,0.051,-123.781884,0.005,-1.800069,0.121,-1.989895,0.088,-2.352853,0.089,-10.294885,0.005,-11.467861,0.002,-13.304661,0.003
AddressQualifier[T.IN STREET REMOTE FROM GAZETTEER LOCATION],71.759995,0.472,69.432902,0.500,66.868047,0.558,2.196983,0.468,2.351071,0.439,1.949553,0.589,33.998655,0.000,32.245066,0.001,29.011645,0.012


,PumpSecondsOnSite_min coefficients,PumpSecondsOnSite_min pvalues,PumpSecondsOnSite_mean coefficients,PumpSecondsOnSite_mean pvalues,PumpSecondsOnSite_max coefficients,PumpSecondsOnSite_max pvalues,TurnoutTimeSeconds_min coefficients,TurnoutTimeSeconds_min pvalues,TurnoutTimeSeconds_mean coefficients,TurnoutTimeSeconds_mean pvalues,TurnoutTimeSeconds_max coefficients,TurnoutTimeSeconds_max pvalues,TravelTimeSeconds_min coefficients,TravelTimeSeconds_min pvalues,TravelTimeSeconds_mean coefficients,TravelTimeSeconds_mean pvalues,TravelTimeSeconds_max coefficients,TravelTimeSeconds_max pvalues,Discord
AddressQualifier[T.IN STREET CLOSE TO GAZETTEER LOCATION],-71.314981,0.094,-111.342187,0.011,-156.308071,0.001,-1.701157,0.188,-2.016420,0.120,-2.592226,0.093,-11.171112,0.006,-11.915869,0.005,-13.563712,0.006,True
AddressQualifier[T.IN STREET OUTSIDE GAZETTEER LOCATION],-32.952347,0.389,-76.945215,0.051,-123.781884,0.005,-1.800069,0.121,-1.989895,0.088,-2.352853,0.089,-10.294885,0.005,-11.467861,0.002,-13.304661,0.003,True
AddressQualifier[T.IN STREET REMOTE FROM GAZETTEER LOCATION],71.759995,0.472,69.432902,0.500,66.868047,0.558,2.196983,0.468,2.351071,0.439,1.949553,0.589,33.998655,0.000,32.245066,0.001,29.011645,0.012,True
AddressQualifier[T.NEARBY ADDRESS - NO BUILDING IN STREET],32.345765,0.662,31.124183,0.684,22.315209,0.792,-1.167781,0.603,-0.635456,0.778,-0.611948,0.820,22.762418,0.001,32.576521,0.000,41.583761,0.000,True
AddressQualifier[T.NEARBY ADDRESS - STREET NOT LISTED IN GAZETTEER],254.978086,0.000,221.198055,0.003,179.649626,0.028,1.658677,0.445,0.894283,0.682,-0.401767,0.877,23.108845,0.001,23.886023,0.001,24.755517,0.003,True


957 discords sur 1440


In [ ]:
# Traitmeent des catégories. Uniquement des catégories nominales
# possibilité Test Anova,


def ShowValues(col):
    print(col, len(df[col].unique()))
    # print(df[col].value_counts())


cols_categ = [
    "StopCode",
    "PropertyType",
    "AddressQualifier",
    "Postcode_district",
    "USRN",
    "IncGeo_BoroughCode",
    "IncGeo_WardCode",
    "FirstPumpArriving_DeployedFromStation",
]

cols_cible = [
    ["PumpSecondsOnSite_min",
    "PumpSecondsOnSite_mean",
    "PumpSecondsOnSite_max"],
    ["TurnoutTimeSeconds_min",
    "TurnoutTimeSeconds_mean",
    "TurnoutTimeSeconds_max"],
    ["TravelTimeSeconds_min",
    "TravelTimeSeconds_mean",
    "TravelTimeSeconds_max"]
]

cols_cible_type = ["PumpSecondsOnSite", "TurnoutTimeSeconds", "TravelTimeSeconds"]

cols_cible_cat = ["FirstPumpArriving_DeployedFromStation", "NumPumpsAttending"]

# prépare les X et y de tests
# X = df.drop(cols_cible, axis=1)
# list_y = []
# for col in cols_cible:
#     list_y.append([col, df[col]])

# nb de valeurs par catégories
for col in cols_categ:
    ShowValues(col)
print()

df_temp = df[df.CalYear > 2020]

# Sélection de caractéristiques récursive
import statsmodels.api
from statsmodels.api import stats
import statsmodels.formula.api as smf

# Anova sur chaque colonne cible avec toutes les colonnes catégorielles
all_result = []
cols_to_check = "+".join(cols_categ)
for cols in cols_cible:
    cible_result = []
    for col in cols:
        print("Test de", col)
        results = smf.ols(f"{col} ~ {cols_to_check}", data=df_temp).fit()
        # table = stats.anova_lm(result)
        # display(table)
        results_df = pd.DataFrame(
            {
                f"{col} coefficients": round(results.params, 3),
                f"{col} pvalues": round(results.pvalues, 3),
            }
        )
        cible_result.append(results_df)
        # display(results_df.head(30))
        # print("Test de", col, "ok")
    all_result.append(cible_result)

for cible_results in all_result:
    for result in cible_results:
        display(result.head(5))

# merge les result dans un dataFrame
compiled_results = []
for cible_results in all_result:
    cible_result = cible_results[0]
    i = 0
    for result in cible_results:
        if i == 0:
            print("continue")
            i += 1
            continue
        print("merge")
        cible_result = pd.merge(
            cible_result, result, left_index=True, right_index=True, how="outer"
        )
    compiled_results.append(cible_result)

# affiche les df mergés par type de cible
# print("Preview")
# for cible_results in compiled_results:
#     display(cible_results.head(3))

# regarde si la pvalue est la même pour toutes les caétgories : ajout de la colonne Discord : True si il y a des delta
# filtre les lignes avec que des valeurs > 0.05 ou que que des valeurs < 0.05
# pour voir si certaines interviennent que dans certains cas
def TreatDiscordRow(columns, row):
    keeps = []
    nb = 0
    for i in range(1, len(columns), 2):
        # print("col", columns[i])
        # print(row)
        keeps.append(row[columns[i]] <= 0.05)
        nb += 1
    val = not (sum(keeps) == 0) | (sum(keeps) == nb)  # sum(keeps)
    # print(val)
    return val

def TreatKeepRow(columns, row):
    keeps = []
    nb = 0
    for i in range(1, len(columns), 2):
        # print("col", columns[i])
        # print(row[columns[i]])
        keeps.append(row[columns[i]] <= 0.05)
        nb += 1
    val = sum(keeps)
    # si toutes les pvalues sont non significatives
    if val == 0:
        return False
    # si toutes les pvalues sont significatives
    elif val == nb:
        return True
    # sinon, conserve si c'est significatif sur la moyenne
    else:
        mean_col = [s for s in columns if "mean pvalues" in s][0]
        val = row[mean_col]
        # print("mean_col", mean_col)
        # print("mean_col", val)
        return True if val <= 0.05 else False
    # print(val)
    return val

print("Calcule Discords")
for cible_results in compiled_results:
    # display(cible_results.head())
    cible_results["Discord"] = cible_results.apply(lambda row: TreatDiscordRow(cible_results.columns, row), axis=1)
    cible_results["Keep"] = cible_results.apply(lambda row: TreatKeepRow(cible_results.columns, row), axis=1)
    display(cible_results[cible_results.Discord == True].head())

for i in range(0, len(compiled_results)):    
    print(
        f"{cols_cible_type[i]} {compiled_results[i]["Discord"].sum()} discords, {compiled_results[i]["Keep"].sum()} keeps, sur {len(compiled_results[i])}, " 
        f"conserve {round(compiled_results[i]["Keep"].sum() / len(compiled_results[i]) * 100, 0)}%"
    )
# 957 discords sur 1440


# pour les temps de mobilisation on considère
# model = smf.ols(
#     formula="TravelTimeSeconds_min ~ StopCode + PropertyType + USRN + HourOfCall",
#     data=df_temp,
# )
# results = model.fit()
# # Résumé des résultats
# display(results.summary())

# model = smf.ols(
#     formula="PumpSecondsOnSite_min ~ StopCode + PropertyType + USRN",
#     data=df_temp,
# )
# results = model.fit()
# results_df = pd.DataFrame({"coefficients": results.params, "pvalues": results.pvalues})
# # Filtrer pour ne garder que les p-values <= 0.05
# significant_results = results_df[results_df["pvalues"] <= 0.05]
# # Afficher les résultats filtrés
# display(significant_results)


StopCode 28
PropertyType 291
AddressQualifier 11
Postcode_district 322
USRN 53339
IncGeo_BoroughCode 33
IncGeo_WardCode 843
FirstPumpArriving_DeployedFromStation 117

Test de PumpSecondsOnSite_min


In [154]:
# sauve les infos des valeurs catégorielles et leur conservation
for i in range(0, len(compiled_results)):
    df_to_save = compiled_results[i][["Keep"]]
    name = f"keep {cols_cible_type[i]}"
    display(df_to_save.head())
    print(name)
    df_to_save.to_csv(f"../data/{name}.csv", sep=";", index=True)

,Keep
AddressQualifier[T.IN STREET CLOSE TO GAZETTEER LOCATION],True
AddressQualifier[T.IN STREET OUTSIDE GAZETTEER LOCATION],False
AddressQualifier[T.IN STREET REMOTE FROM GAZETTEER LOCATION],False
AddressQualifier[T.NEARBY ADDRESS - NO BUILDING IN STREET],False
AddressQualifier[T.NEARBY ADDRESS - STREET NOT LISTED IN GAZETTEER],True


keep PumpSecondsOnSite


,Keep
AddressQualifier[T.IN STREET CLOSE TO GAZETTEER LOCATION],False
AddressQualifier[T.IN STREET OUTSIDE GAZETTEER LOCATION],False
AddressQualifier[T.IN STREET REMOTE FROM GAZETTEER LOCATION],False
AddressQualifier[T.NEARBY ADDRESS - NO BUILDING IN STREET],False
AddressQualifier[T.NEARBY ADDRESS - STREET NOT LISTED IN GAZETTEER],False


keep TurnoutTimeSeconds


,Keep
AddressQualifier[T.IN STREET CLOSE TO GAZETTEER LOCATION],True
AddressQualifier[T.IN STREET OUTSIDE GAZETTEER LOCATION],True
AddressQualifier[T.IN STREET REMOTE FROM GAZETTEER LOCATION],True
AddressQualifier[T.NEARBY ADDRESS - NO BUILDING IN STREET],True
AddressQualifier[T.NEARBY ADDRESS - STREET NOT LISTED IN GAZETTEER],True


keep TravelTimeSeconds


In [ ]:
# Vérification de la pertinence des colonnes CalYear HourOfCall Month DayOfWeek

"CalYear"	"HourOfCall"
"Month"	"DayOfWeek"

df_temp = df[df.CalYear > 2020]

# Anova sur chaque colonne cible avec toutes les colonnes catégorielles
all_result = []
cols_to_check = "+".join(cols_categ)
for cols in cols_cible:
    cible_result = []
    for col in cols:
        print("Test de", col)
        results = smf.ols(f"{col} ~ {cols_to_check}", data=df_temp).fit()
        # table = stats.anova_lm(result)
        # display(table)
        results_df = pd.DataFrame(
            {
                f"{col} coefficients": round(results.params, 3),
                f"{col} pvalues": round(results.pvalues, 3),
            }
        )
        cible_result.append(results_df)
        # display(results_df.head(30))
        # print("Test de", col, "ok")
    all_result.append(cible_result)

for cible_results in all_result:
    for result in cible_results:
        display(result.head(10))

In [6]:
# supprime IncidentNumber qui n'est plus nécessaire et ne doit pas être une donnée d'entrainement
#df = df.drop(["IncidentNumber"], axis=1)

In [ ]:
merge.to_csv("../data/PreProcess.csv", sep=";", index=False)